# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

In [3]:
# Modification in DB

cursor = conn.cursor()
cursor.execute('UPDATE Movie SET year = REPLACE(year, "I", "");')
cursor.execute('UPDATE Movie SET year = REPLACE(year, "V", "");')
cursor.execute('UPDATE Movie SET year = REPLACE(year, "X ", "");')
cursor.execute('UPDATE Movie SET title = LTRIM(title);')
cursor.execute('UPDATE Movie SET year = RTRIM(LTRIM(year));')
cursor.execute('UPDATE Movie SET rating = RTRIM(LTRIM(rating));')
cursor.execute('UPDATE Movie SET num_votes = RTRIM(LTRIM(num_votes));')

cursor.execute('UPDATE M_Producer SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Producer SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Director SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Director SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Cast SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Cast SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Genre SET gid = RTRIM(LTRIM(gid));')
cursor.execute('UPDATE M_Genre SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE Genre SET gid = RTRIM(LTRIM(gid));')
cursor.execute('UPDATE Genre SET name = RTRIM(LTRIM(name));')

cursor.execute('UPDATE Person SET name = RTRIM(LTRIM(name));')
cursor.execute('UPDATE Person SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE Person SET gender = RTRIM(LTRIM(gender));')

conn.commit()

## Sample Code

In [4]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
Wall time: 97.8 ms


,Movie_Name
0,Mastizaade
1,Dragonball Evolution
2,Loveyatri
3,Race 3
4,Gunday


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [5]:
%%time
# Write your sql query below

query = """
        SELECT P.Name AS 'Director', M.title, M."year"
        FROM Person P 
        JOIN M_Director MD ON MD.PID = P.PID 
        JOIN Movie M ON M.MID = MD.MID 
        JOIN M_Genre MG ON MG.MID = M.MID
        JOIN Genre G ON G.GID = MG.GID 
        WHERE G.Name LIKE '%Comedy%'
        AND M."year" % 4 = 0
        """

q1 = pd.read_sql_query(query, conn)    
print(q1.shape)
q1.head()

(232, 3)
Wall time: 80.8 ms


,Director,title,year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [6]:
%%time
# Write your sql query below

query = """
        SELECT P.Name
        FROM Person P 
        JOIN M_Cast MC ON P.PID = TRIM(MC.PID) 
        JOIN Movie M ON M.MID = MC.MID 
        WHERE M.title = 'Anand' AND M."year" = 1971
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(17, 1)
Wall time: 215 ms


,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Brahm Bhardwaj
3,Ramesh Deo
4,Seema Deo


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [7]:
%%time
# Write your sql query below

query = """
        WITH 
        CAST_ON_1970 AS (SELECT TRIM(P.PID) FROM Person P JOIN M_Cast MC ON P.PID = TRIM(MC.PID) JOIN Movie M ON M.MID = MC.MID WHERE M."year" < 1970),
        CAST_ON_1990 AS (SELECT TRIM(P.PID) FROM Person P JOIN M_Cast MC ON P.PID = TRIM(MC.PID) JOIN Movie M ON M.MID = MC.MID WHERE M."year" > 1990)

        SELECT P.Name AS 'Actor'
        FROM Person P
        WHERE TRIM(PID) IN CAST_ON_1970 AND TRIM(P.PID) IN CAST_ON_1990
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
q3.head()

(300, 1)
Wall time: 610 ms


,Actor
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [8]:
%%time
# Write your sql query below

query = """
        WITH 
        DIRECTOR_MOVIES AS (SELECT MD.PID, COUNT(*) AS Movie_Count FROM M_Director MD GROUP BY MD.PID HAVING COUNT(*) > 10)
        
        SELECT P.Name, DM.Movie_Count 
        FROM Person P 
        JOIN DIRECTOR_MOVIES DM ON P.PID = DM.PID;
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
q4.head()

(45, 2)
Wall time: 19.9 ms


,Name,Movie_Count
0,Mahesh Manjrekar,15
1,Satish Kaushik,12
2,Anurag Kashyap,13
3,Yash Chopra,21
4,Subhash Ghai,18


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [9]:
%%time
# Write your sql query below

query = """
        WITH 
        FEMALE_ONLY_MOVIES AS (SELECT MC.MID FROM M_Cast MC INNER JOIN Person P ON P.PID = TRIM(MC.PID) WHERE P.Gender = 'Male')

        SELECT M."year", COUNT(*) AS 'count'
        FROM Movie M
        WHERE M.MID not in FEMALE_ONLY_MOVIES
        GROUP BY M."year"
        ORDER BY M."year"
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(6, 2)
Wall time: 215 ms


,year,count
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1


In [10]:
%%time
# Write your sql query below

query = """ 
        WITH
        FEMALE_ONLY_MOVIES AS (SELECT MC.MID FROM M_Cast MC INNER JOIN Person P ON P.PID = TRIM(MC.PID) WHERE P.Gender = 'Male')

        SELECT M."year", P.Name, P.Gender, M.title
        FROM Movie M
        JOIN M_Cast MC ON TRIM(MC.MID) = M.MID
        JOIN Person P ON P.PID = TRIM(MC.PID)
        WHERE M.MID NOT IN FEMALE_ONLY_MOVIES
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a

(25, 4)
Wall time: 371 ms


,year,Name,Gender,title
0,2018,Pihu Myra Vishwakarma,Female,Pihu
1,2018,Prerna Vishwakarma,Female,Pihu
2,2000,Jyothika,Female,Snegithiye
3,2000,Sharbani Mukherji,Female,Snegithiye
4,2000,Tabu,Female,Snegithiye
5,2000,Lakshmi,Female,Snegithiye
6,2000,Ishita Arun,Female,Snegithiye
7,2000,Sukumari,Female,Snegithiye
8,2000,Aachi Manorama,Female,Snegithiye
9,2000,Bharti Achrekar,Female,Snegithiye


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [11]:
%%time
# Write your sql query below

query = """
        WITH 
        MALE_MOVIES AS (SELECT MC.MID FROM M_Cast MC INNER JOIN Person P ON P.PID = TRIM(MC.PID) WHERE P.Gender = 'Male'),
        FEMALE_MOVIES AS (SELECT M."year", COUNT(*) AS 'movies_count' FROM Movie M WHERE M.MID NOT IN MALE_MOVIES GROUP BY M."year"),
        ALL_YEARS AS (SELECT M."year", COUNT(*) AS 'total_movies' FROM Movie as M GROUP BY M."year")

        SELECT FM."year", AY.total_movies, ((FM.movies_count * 100.0) / AY.total_movies) AS Percent
        FROM FEMALE_MOVIES FM 
        INNER JOIN ALL_YEARS AY ON FM."year" = AY."year"
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
q5b.head()

(6, 3)
Wall time: 189 ms


,year,total_movies,Percent
0,1939,2,50.000000
1,1999,66,1.515152
2,2000,64,1.562500
3,2009,110,0.909091
4,2012,111,0.900901


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [12]:
%%time
# Write your sql query below

query = """
        WITH 
        LARGER_CAST AS (SELECT COUNT(*) AS CAST_COUNT, MID FROM M_Cast GROUP BY MID)
        
        SELECT LC.MID, M.TITLE, MAX(LC.CAST_COUNT) AS CAST_SIZE
        FROM LARGER_CAST LC
        JOIN Movie M ON M.MID = LC.MID 
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
q6.head()

(1, 3)
Wall time: 57.8 ms


,MID,title,CAST_SIZE
0,tt5164214,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [13]:
%%time
# Write your sql query below

query = """
        WITH 
        UNIQUE_YEAR AS (SELECT DISTINCT "year" FROM Movie)
        
        SELECT D."year" AS START, D."year"+9 AS END, COUNT(*)  
        FROM UNIQUE_YEAR D
        JOIN Movie M on M."year" >= START AND M."year"<= END
        GROUP BY END 
        ORDER BY COUNT(*) DESC 
        LIMIT 1
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
q7.head()

(1, 3)
Wall time: 98.8 ms


,START,END,COUNT(*)
0,2008,2017,1205


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [14]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(P.Name) AS ACTOR_NAME,  COUNT(DISTINCT M.MID) AS YASH_CHOPRA_DIRECTED_MOVIES
        FROM Person P 
        JOIN M_Cast MC ON TRIM(MC.PID) = P.PID 
        JOIN Movie M ON M.MID = MC.MID 
        JOIN M_Director MD ON MD.MID = M.MID 
        JOIN Person P1 ON P1.PID = TRIM(MD.PID)
        WHERE TRIM(P1.Name) = 'Yash Chopra'
        GROUP BY TRIM(P.PID)
        ORDER BY COUNT(DISTINCT M.MID) DESC
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head()

(430, 2)
Wall time: 445 ms


,ACTOR_NAME,YASH_CHOPRA_DIRECTED_MOVIES
0,Jagdish Raj,11
1,Manmohan Krishna,10
2,Iftekhar,9
3,Madan Puri,8
4,Vikas Anand,8


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [15]:
%%time
# Write your sql query below

query = """
        WITH 
        Shah_rukh_khan_index AS (SELECT  P.PID, MC.MID, P.Name, ROW_NUMBER() OVER (PARTITION BY MC.MID) AS ROW_NUM FROM  Person P JOIN M_Cast MC ON TRIM(MC.PID) = P.PID)

        SELECT P.Name, MC.MID, M.title 
        FROM Person P 
        JOIN M_Cast MC ON TRIM(MC.PID) = P.PID 
        JOIN Movie M ON M.MID = TRIM(MC.MID)
        WHERE M.MID IN (SELECT MID FROM Shah_rukh_khan_index WHERE Name LIKE 'Shah Rukh Khan%' and ROW_NUM = 2)
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(535, 3)
Wall time: 649 ms


,Name,MID,title
0,Alia Bhatt,tt5946128,Dear Zindagi
1,Shah Rukh Khan,tt5946128,Dear Zindagi
2,Kunal Kapoor,tt5946128,Dear Zindagi
3,Priyanka Moodley,tt5946128,Dear Zindagi
4,Prateek Jain,tt5946128,Dear Zindagi
